In [1]:
import numpy as np
from pyscf import gto, scf, ao2mo

$$
\begin{split}
J_{\kappa \lambda} & = \sum_{i=1}^N  \sum_{\mu}^{M} \sum_{\nu}^{M} c_{i, \mu} c_{i, \nu} [\mu \nu | \kappa \lambda]\\
J_{\kappa \lambda} & = \sum_{i=1}^N  [\phi_i \phi_i | \kappa \lambda]
\end{split}
$$

In [2]:
def build_J(Fouridx,C,N):
    J = np.zeros(C.shape)
    for kappa in range(0,Fouridx.shape[0]):
        for lamda in range(0,Fouridx.shape[0]):
            for i in range(0,N): # all occupied
                for mu in range(0,Fouridx.shape[0]): # all basis functions
                    for nu in range(0,Fouridx.shape[0]): # all basis functions
                        J[kappa][lamda] += C[mu,i]*C[nu,i]*Fouridx[mu,nu,kappa,lamda]
    return 2*J # needs factor 2 because its spin restricted

$$
\begin{split}
J_{\kappa \lambda} & = \sum_{i=1}^N  \sum_{\mu}^{M} \sum_{\nu}^{M} c_{i, \mu} c_{i, \nu} [\mu  \kappa| \nu  \lambda]\\
J_{\kappa \lambda} & = \sum_{i=1}^N  [\phi_i \kappa | \phi_i \lambda]
\end{split}
$$

In [3]:
def build_K(Fouridx,C,N):
    K = np.zeros(C.shape)
    for kappa in range(0,Fouridx.shape[0]):
        for lamda in range(0,Fouridx.shape[0]):
            for i in range(0,N): # all occupied
                for mu in range(0,Fouridx.shape[0]): # all basis functions
                    for nu in range(0,Fouridx.shape[0]): # all basis functions
                        K[kappa][lamda] += C[mu,i]*C[nu,i]*Fouridx[mu,kappa,nu,lamda]
    return 2*K # needs factor 2 because its spin restricted

In [4]:
mol = gto.Mole()
mol.atom = """
    Ne    0.    0.    0.
"""
# this basis has 2 functions for Helium
mol.basis = "6-31g" #mol.basis = "ccpvdz", mol.basis = "sto-6g"
mol.build()

# the 2 electron integrals \langle \mu \nu | \kappa \lambda \rangle have M^4 in the case of  case 16 distinct elements
eri = mol.intor('int2e')

In [5]:
mf = scf.RHF(mol)
mf.kernel()
J = mf.get_j()
K = mf.get_k()
C = mf.mo_coeff
N = mol.nelec[0]
P=np.matmul(C[:,0:N],C[:,0:N].T)

converged SCF energy = -128.473876870668


In [6]:
print(J)

[[ 2.34250003e+01  4.30815331e+00  3.26823406e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 4.30815331e+00  1.24357919e+01  8.60782026e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 3.26823406e+00  8.60782026e+00  8.83395486e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.28730330e+01
   4.65603452e-18 -2.70308600e-17  4.93165923e+00  2.65850295e-18
  -8.67796140e-18]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  4.65603452e-18
   1.28730330e+01  5.18284322e-18  2.65850295e-18  4.93165923e+00
   9.71061069e-19]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -2.70308600e-17
   5.18284322e-18  1.28730330e+01 -8.67796140e-18  9.71061069e-19
   4.93165923e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  4.93165923e+00
   2.65850295e-18 -8.6779614

In [7]:
J_2 = build_J(eri,C,N)
print(J_2)

[[ 2.34250003e+01  4.30815331e+00  3.26823406e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 4.30815331e+00  1.24357919e+01  8.60782026e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 3.26823406e+00  8.60782026e+00  8.83395486e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.28730330e+01
   1.30104261e-18 -2.94360890e-17  4.93165923e+00  1.51788304e-18
  -9.40545385e-18]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.30104261e-18
   1.28730330e+01  4.66206934e-18  1.51788304e-18  4.93165923e+00
   6.84402621e-19]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -2.94360890e-17
   4.66206934e-18  1.28730330e+01 -9.40545385e-18  6.84402621e-19
   4.93165923e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  4.93165923e+00
  -2.16840434e-19 -9.1886134

In [8]:
print(J[0,0]/J_2[0,0], J[1,1]/J_2[1,1])

1.0 1.0000000000000009


In [9]:
print(K)

[[ 1.24802204e+01  2.01345026e+00  1.64951921e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 2.01345026e+00  3.65352126e+00  2.98783781e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 1.64951921e+00  2.98783781e+00  2.64819848e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  3.08225045e+00
   3.64041271e-17 -2.51779758e-16  1.68027926e+00  3.36922660e-17
  -1.09566629e-16]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  3.64041271e-17
   3.08225045e+00  5.22242501e-17  3.73788136e-17  1.68027926e+00
  -1.01176268e-17]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -2.51779758e-16
   5.22242501e-17  3.08225045e+00 -6.33950873e-17  1.80731148e-17
   1.68027926e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.68027926e+00
   3.73788136e-17 -6.3395087

In [10]:
K_2 = build_K(eri,C,N)
print(K_2)

[[ 1.24802204e+01  2.01345026e+00  1.64951921e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 2.01345026e+00  3.65352126e+00  2.98783781e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 1.64951921e+00  2.98783781e+00  2.64819848e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  3.08225045e+00
   1.16009632e-17 -2.41831295e-16  1.68027926e+00  1.36609474e-17
  -1.04381564e-16]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -1.38777878e-17
   3.08225045e+00  4.45064992e-17  1.73472348e-17  1.68027926e+00
  -8.97854924e-18]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -2.38958159e-16
   5.69206141e-17  3.08225045e+00 -6.85215773e-17  1.98951099e-17
   1.68027926e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.68027926e+00
   2.10335221e-17 -6.8440262